<a href="https://colab.research.google.com/github/jchoi92k/CS8980/blob/main/Homework%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Create a cosine similarity matrix for all of Shakespeare’s works found in the provided
file. This will result in a 42 by 42 matrix with the cosine similarity between each of his
works. In other words, calculate the document-wise cosine similarity between all of
Shakepeare’s works. (50 points). Use TF_IDF for this. Note, you can use the Cosine
Similarity function on scikit-learn or implement your own, but no other library/package is
allowed.


In [ ]:
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# All files were uploaded to a folder named 'shakespeare'
textpaths = [os.path.join('shakespeare', w) for w in os.listdir('shakespeare')]
texts = []

# Open files
for textpath in textpaths:
  with open(textpath, 'r', encoding='utf-8') as f:
    content = f.read()
    texts.append(content)

In [ ]:
# TF-IDF vectorization
vct = TfidfVectorizer()
fitted = vct.fit_transform(texts)

# Compute cosine similarity
cosim = cosine_similarity(fitted, fitted)
print(cosim)

[[1.         0.59813572 0.65459176 ... 0.52515851 0.57395169 0.69694474]
 [0.59813572 1.         0.66801609 ... 0.5890345  0.59258345 0.71771293]
 [0.65459176 0.66801609 1.         ... 0.59895863 0.66716633 0.79038899]
 ...
 [0.52515851 0.5890345  0.59895863 ... 1.         0.52277672 0.62033426]
 [0.57395169 0.59258345 0.66716633 ... 0.52277672 1.         0.70530628]
 [0.69694474 0.71771293 0.79038899 ... 0.62033426 0.70530628 1.        ]]


In [409]:
# To show shape
cosim.shape

(42, 42)

2. Write a function that takes the previous matrix and a number n as parameters (nothing
else will be accepted) and return the top n similar works. Use the function to output the
top 10 similar works. (30 points).


In [ ]:
import pandas as pd

# Takes matrix and number n as arguments and returns indices of top n similar works in descending order
def topnsim(prevmat, n):
  # Create dataframe with matrix to sort the cosine similarities
  rows = []
  cols = []
  for row in range(len(prevmat)):
    rows += [row] * len(prevmat)
  for col in range(len(prevmat[0])):
    cols.append(col)
  cols = cols * len(prevmat[0])
  vals = prevmat.flatten()
  df = pd.DataFrame({'row_index':rows, 'col_index':cols, 'values':vals})

  # Remove all 1.0 cosine-similarities and sort (descending)
  df = df[df['values'] < 0.999999]
  df = df.sort_values(by=['values'], ascending=False)

  # Append results to this list
  res = []

  # Skip every other row because they are necessarily duplicates
  for x, y in zip(df['row_index'][::2], df['col_index'][::2]):
    if len(res) < 10:
      res.append((x, y))
  return res

# Get indices using the function defined above
indices = topnsim(cosim, 10)

# Print similar works using indices
filenames = os.listdir('shakespeare')
for indice in indices:
  print(f'"{filenames[indice[0]]}" is similar to "{filenames[indice[1]]}"')

"lucrece_TXT_FolgerShakespeare.txt" is similar to "venus-and-adonis_TXT_FolgerShakespeare.txt"
"henry-iv-part-1_TXT_FolgerShakespeare.txt" is similar to "henry-iv-part-2_TXT_FolgerShakespeare.txt"
"lucrece_TXT_FolgerShakespeare.txt" is similar to "shakespeares-sonnets_TXT_FolgerShakespeare.txt"
"henry-vi-part-1_TXT_FolgerShakespeare.txt" is similar to "henry-vi-part-2_TXT_FolgerShakespeare.txt"
"richard-iii_TXT_FolgerShakespeare.txt" is similar to "richard-ii_TXT_FolgerShakespeare.txt"
"venus-and-adonis_TXT_FolgerShakespeare.txt" is similar to "shakespeares-sonnets_TXT_FolgerShakespeare.txt"
"henry-v_TXT_FolgerShakespeare.txt" is similar to "henry-viii_TXT_FolgerShakespeare.txt"
"henry-vi-part-2_TXT_FolgerShakespeare.txt" is similar to "henry-vi-part-3_TXT_FolgerShakespeare.txt"
"henry-vi-part-2_TXT_FolgerShakespeare.txt" is similar to "richard-ii_TXT_FolgerShakespeare.txt"
"king-john_TXT_FolgerShakespeare.txt" is similar to "henry-v_TXT_FolgerShakespeare.txt"


3. Using the code from the Language Models II class, train two simple language models
using all of the files (together) in shakespeares-works_TXT_FolgerShakespeare.zip. One
model should be trained using bigrams, the other using trigrams. (40 points).

In [ ]:
import nltk
from nltk import bigrams, trigrams, sent_tokenize, word_tokenize
from collections import Counter, defaultdict

nltk.download('punkt')

# Tokenize sentences
tokenized_texts = []

for text in texts:
  tokenized_sents = sent_tokenize(text)
  for sent in tokenized_sents:
    tokenized_words = word_tokenize(sent)
    tokenized_texts.append(tokenized_words)

# For a trigram model => model
model = defaultdict(lambda: defaultdict(lambda: 0))

for sentence in tokenized_texts:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

# For a bigram model => model2
model2 = defaultdict(lambda: defaultdict(lambda: 0))

for sentence in tokenized_texts:
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        model2[w1][w2] += 1
 
for w1 in model2:
    total_count = float(sum(model2[w1].values()))
    for w2 in model2[w1]:
        model2[w1][w2] /= total_count

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


4. Write a function that takes the following three parameters: model, list of start words,
number of sentences to generate. This function should return the sentences generated
as a list. DO NOT print anything to the screen from within the function. Use this function
to generate 10 sentences with the bigram model from the previous question, and 5
sentences with the trigram model from the previous question. (30 points)

In [ ]:
import random

def generate_sentence(mdl, list_of_start_word, nofsent):
  # Append results here
  res = []

  # Find out if model is a bigram model or a trigram model
  ngram_mode = ''
  if type(list(mdl)[0]) == tuple:
    ngram_mode = 'trigram'
  else:
    ngram_mode = 'bigram'

  # Generate sentences
  for _ in range(nofsent):
    newsent = list_of_start_word.copy()
    sentence_finished = False
    while not sentence_finished:
      # select a random probability threshold  
      r = random.random()
      accumulator = .0

      # For trigrams
      if ngram_mode == 'trigram':
        for word in mdl[tuple(newsent[-2:])].keys():
            accumulator += mdl[tuple(newsent[-2:])][word]
            # select words that are above the probability threshold
            if accumulator >= r:
                newsent.append(word)
                break
        if newsent[-2:] == [None, None]:
            sentence_finished = True

      # For bigrams
      elif ngram_mode == 'bigram':
        for word in mdl[newsent[-1]].keys():
            accumulator += mdl[newsent[-1]][word]
            # select words that are above the probability threshold
            if accumulator >= r:
                newsent.append(word)
                break
        if newsent[-1:] == [None]:
            sentence_finished = True

    generated = ' '.join([t for t in newsent if t])
    res.append(generated)
    
  return res

In [ ]:
# 10 sentences with the bigram model
random.seed(15)
generate_sentence(model2, ['and'], 10)

["and sprinkles in faith , Deposing thee speak more exalted with his surname , To th ' offended you believe me must think you have I am king ?",
 'and wagered with the custard ; I attach you , And scarce serve thee .',
 'and learn , for the highest mount .',
 "and whose souls ; and yet a desire of Caesar 's love , Soul-killing witches that I do my falsehood with feet I am proof hath power to know One that will stir !",
 "and finally , He 's the bones .",
 'and his grave ?',
 'and both the request a card or to obey the map .',
 'and loving king , lieutenant .',
 'and that there ?',
 "and bridegroom 's shot a maid again ."]

In [ ]:
# 5 sentences with the trigram model
random.seed(15)
generate_sentence(model, ['and', 'the'], 5)

["and the King's Tavern .",
 'and the prisoner ?',
 "and the greatness of his valor can not tell -- But Banquo 's safe ?",
 'and the blows you gave him when Hector has knocked out , Must fetch him .',
 'and the thing he would change places with his hat ; it is no such thing , what then ?']

Bonus (20 points): Using the same methodology from questions 1 and 2, create a
similarity matrix between the 20 newsgroups corpus. And find the top 5 similar
newsgroups

In [ ]:
import numpy as np
import os
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Fetch data
data = fetch_20newsgroups()

In [ ]:
# Create empty dictionary to sort different newsgroup categories
emdic = defaultdict(str)

for w, k in zip(data.data, data.target):
  emdic[k] = emdic[k] + "\n" + w
newstexts = []

for key in sorted(emdic.keys()):
  newstexts.append(emdic[key])

# TF-IDF vectorization
vct_n = TfidfVectorizer()
fitted_n = vct.fit_transform(newstexts)

# Compute cosine similarity
cosim_n = cosine_similarity(fitted_n, fitted_n)
print(cosim_n)

[[1.         0.9094271  0.05523086 0.88417914 0.90353323 0.89906611
  0.77552111 0.93001747 0.9058185  0.90239171 0.8587065  0.92177557
  0.93163157 0.94660545 0.91830139 0.97013249 0.9449579  0.91569666
  0.95436217 0.97017371]
 [0.9094271  1.         0.05796562 0.91413931 0.92825112 0.92834378
  0.85646081 0.92462808 0.90894411 0.89050075 0.85412391 0.91324122
  0.94087947 0.93113063 0.92194395 0.90809985 0.91542406 0.88169114
  0.91564806 0.91395076]
 [0.05523086 0.05796562 1.         0.05744383 0.05733129 0.056494
  0.05259049 0.05673589 0.05597304 0.05472295 0.05218132 0.05595068
  0.05774222 0.05614403 0.05536226 0.05484025 0.05619588 0.05269622
  0.0554367  0.05510923]
 [0.88417914 0.91413931 0.05744383 1.         0.94475744 0.90136798
  0.83384539 0.91845189 0.90008117 0.87931077 0.84843403 0.89333141
  0.92985151 0.89963781 0.89441629 0.88508149 0.89747512 0.86216335
  0.89789488 0.89203503]
 [0.90353323 0.92825112 0.05733129 0.94475744 1.         0.91681064
  0.84018088 0.938

In [ ]:
# Get indices using the function defined above
indices_n = topnsim(cosim_n, 5)

# Print similar works using indices
for indice in indices_n:
  print(f'"{data.target_names[indice[0]]}" is similar to "{data.target_names[indice[1]]}"')

"soc.religion.christian" is similar to "talk.religion.misc"
"talk.politics.guns" is similar to "talk.politics.misc"
"talk.religion.misc" is similar to "alt.atheism"
"alt.atheism" is similar to "soc.religion.christian"
"talk.religion.misc" is similar to "talk.politics.misc"
"talk.politics.misc" is similar to "soc.religion.christian"
"talk.politics.guns" is similar to "talk.religion.misc"
"rec.autos" is similar to "sci.electronics"
"soc.religion.christian" is similar to "talk.politics.guns"
"talk.politics.misc" is similar to "alt.atheism"
